<a href="https://colab.research.google.com/github/aruaru0/colab_notebook/blob/main/baseline_train_amp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Aug 14 00:22:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --disable-pip-version-check --no-cache-dir ./


Writing setup.sh


In [3]:
#import os
#assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [4]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [5]:
!tar xvf /content/drive/MyDrive/datas/data/Label.tar

label/
label/val_finetune.txt
label/train_finetune.txt


In [6]:
#!tar xf /content/drive/MyDrive/datas/data/Test.tar
!tar xf /content/drive/MyDrive/datas/data/Train.tar
!tar xf /content/drive/MyDrive/datas/data/Val.tar

# restart

In [7]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 8102, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 8102 (delta 92), reused 124 (delta 63), pack-reused 7913
Receiving objects: 100% (8102/8102), 14.16 MiB | 25.21 MiB/s, done.
Resolving deltas: 100% (5491/5491), done.
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/pyt

In [8]:
!pip install timm

     |████████████████████████████████| 376 kB 16.0 MB/s 


In [9]:
# imports pytorch
#import torch

# imports the torch_xla package
#import torch_xla
#import torch_xla.core.xla_model as xm
#import torch_xla.utils.serialization as xser

In [10]:
import numpy as np
import pandas as pd
import timm
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from PIL import Image
from torchvision import transforms
import torchvision

import tqdm

from PIL import Image

from apex import amp, optimizers



In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #(1)
print(device)

cuda:0


In [12]:
#device = xm.xla_device()
#print(device)

# setting

In [13]:
class CONFIG :
  TRAIN = '/content/label/train_finetune.txt'
  VALID = '/content/label/val_finetune.txt'
  num_of_worker = 4
  base_model = 'resnetrs270'
  batch_size = 32
  num_epochs = 20
  lr = 0.0005

In [14]:
#net = timm.create_model(CONFIG.base_model, pretrained=True)

In [15]:
#net

In [16]:
df_train = pd.read_table(CONFIG.TRAIN, sep=' ', header=None, names=['file','class'])
df_valid = pd.read_table(CONFIG.VALID, sep=' ', header=None, names=['file','class'])

In [17]:
df_train.head()

,file,class
0,0/16330004.jpg,0
1,0/16330005.jpg,0
2,0/16330007.jpg,0
3,0/16330009.jpg,0
4,0/16330010.jpg,0


In [18]:
df_valid.head()

,file,class
0,0/16330020.jpg,0
1,0/16330024.jpg,0
2,0/16330029.jpg,0
3,0/16330037.jpg,0
4,0/16330046.jpg,0


# data loader

In [19]:
class LFFdataset(Dataset):
    def __init__(self, path, df, transform = None):
        self.transform = transform
        self.df = df
        self.path = path
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file = self.path + self.df['file'][idx] 
        #画像読み込み。
        img = Image.open(file)

        #transform事前処理実施
        if self.transform is not None:
            img = self.transform(img)
        label=self.df['class'][idx]

        return img,label
 

# Dataloader

In [20]:
size = 256

transform_train = transforms.Compose([
                                      transforms.ToTensor(),
#                                      transforms.RandomResizedCrop(size, scale=(0.3, 1.0), ratio=(3 / 4, 4 / 3), interpolation=2),
                                      transforms.RandomResizedCrop(size, scale=(0.25, 1.0), ratio=(1, 1), interpolation=2),
#                                      transforms.Resize((400,400)), 
#                                      transforms.RandomHorizontalFlip(),
#                                      transforms.RandomVerticalFlip(),
#                                      transforms.RandomErasing(),
                                      transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
#                                      transforms.Resize((400,400)), 
                                      transforms.Normalize(
                                          mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225]
                                      )
                                      ])

transform_valid = transforms.Compose([
                                      transforms.Resize((size, size)), 
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225]
                                      ),
                                          ])

dataset_train = LFFdataset('./Train/', df_train, transform_train)
dataset_valid = LFFdataset('./Val/', df_valid, transform_valid)
dataloader = dict()
dataloader['train'] = DataLoader(dataset_train, batch_size=CONFIG.batch_size, shuffle=True, num_workers=CONFIG.num_of_worker)
dataloader['valid'] = DataLoader(dataset_valid, batch_size=CONFIG.batch_size, shuffle=False, num_workers=CONFIG.num_of_worker)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:835: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [21]:
#データローダの中身確認
for img,label in dataloader['train']:
    print('label=',label)
    print('img.shape=',img.size())
    break

label= tensor([211, 483,  99, 424, 246, 933, 456, 101, 761, 782, 682, 424, 225, 880,
        233, 635, 815, 345, 623, 965, 122, 759, 455, 320, 275, 303, 499, 439,
        503,  64, 151, 633])
img.shape= torch.Size([32, 3, 256, 256])


# model

In [22]:
class LFFmodel(nn.Module):
    global CONFIG
    def __init__(self):
        super(LFFmodel, self).__init__()
        
        self.net = timm.create_model(CONFIG.base_model, pretrained=True)

    def forward(self, x):
        x = self.net(x)
        return x

# train

In [23]:
if 0 :
  # APEX から AMP をインポートして
  from apex import amp, optimizers

  # AMP を初期化して
  model, optimizer = amp.initialize(model, optimizer, opt_level=args.opt_level, keep_batchnorm_fp32=args.keep_batchnorm_fp32, loss_scale=args.loss_scale)

  # ロススケーリングして逆伝播
  with amp.scale_loss(loss, optimizer) as scaled_loss:
      scaled_loss.backward() 

In [ ]:
model = LFFmodel()
model = model.to(device)

PATH = '/content/drive/MyDrive/datas/last_resnetrs270.pth'
BESTPATH = '/content/drive/MyDrive/datas/best_resnetrs270.pth'
#model.load_state_dict(torch.load(PATH), map_location=torch.device("cpu")))
#model.load_state_dict(torch.load(PATH))

#損失関数（criterion）、最適化関数を定義（optimizer）
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=CONFIG.lr)
#optimizer = optim.Adam(model.net.classifier.parameters(), lr=0.0001)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1")

if 1:
  print("read trained data................")
  checkpoint = torch.load(PATH)
  #model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
  model.load_state_dict(checkpoint['model'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  amp.load_state_dict(checkpoint['amp'])
  print("done.", flush=True)

for e in model.parameters() :
  e.requires_grad = True
for e in model.net.fc.parameters() :
  e.requires_grad = True
#for e in model.net.classifier.parameters() :
#  e.requires_grad = True

best = 0.0

for epoch in range(CONFIG.num_epochs):
    print('-------------', flush=True)
    print('Epoch {}/{}'.format(epoch+1, CONFIG.num_epochs), flush=True)
    
    for phase in ['train', 'valid']:
        
        if phase == 'train':
            # モデルを訓練モードに設定
            model.train()
            N = len(df_train)//CONFIG.batch_size
        else:
            # モデルを推論モードに設定
            model.eval()
            N = len(df_valid)//CONFIG.batch_size
        
        # 損失和
        epoch_loss = 0.0
        # 正解数
        epoch_corrects = 0
        

        # DataLoaderからデータをバッチごとに取り出す
        for inputs, labels in tqdm.tqdm(dataloader[phase], total = N) :
            inputs = inputs.to(device)
            labels = labels.to(device)

            # optimizerの初期化
            optimizer.zero_grad()
            
            # 学習時のみ勾配を計算させる設定にする
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                # 損失を計算
                loss = criterion(outputs, labels)
                
                # ラベルを予測
                _, preds = torch.max(outputs, 1)
                
                # 訓練時はバックプロパゲーション
                if phase == 'train':
                    # 逆伝搬の計算
                    #loss.backward()
                    with amp.scale_loss(loss, optimizer) as scaled_loss:
                      scaled_loss.backward() 
                    # パラメータの更新
                    optimizer.step()
                    #xm.optimizer_step(optimizer, barrier=True)

                
                # イテレーション結果の計算
                # lossの合計を更新
                # PyTorchの仕様上各バッチ内での平均のlossが計算される。
                # データ数を掛けることで平均から合計に変換をしている。
                # 損失和は「全データの損失/データ数」で計算されるため、
                # 平均のままだと損失和を求めることができないため。
                epoch_loss += loss.detach().cpu().item() * inputs.size(0)

                # 正解数の合計を更新
                epoch_corrects += torch.sum(preds == labels.data)

        # epochごとのlossと正解率を表示
        epoch_loss = epoch_loss / len(dataloader[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(dataloader[phase].dataset)

        if phase == 'train':
          # Save checkpoint
          checkpoint = {
              'model': model.state_dict(),
              'optimizer': optimizer.state_dict(),
              'amp': amp.state_dict()
          }
          torch.save(checkpoint, PATH)
          #PATH = '/content/drive/MyDrive/datas/last_resnetrs152.pth'
          #torch.save(model.state_dict(), PATH)
          #xm.save(model.state_dict(), PATH)
        else :          
          if best < epoch_acc :
            best = epoch_acc 
            # Save checkpoint
            checkpoint = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'amp': amp.state_dict()
            }
            torch.save(checkpoint, BESTPATH)

        print("")
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rs-weights/resnetrs270_ema-b40e674c.pth" to /root/.cache/torch/hub/checkpoints/resnetrs270_ema-b40e674c.pth


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
read trained data................
done.
-------------
Epoch 1/20


 23%|██▎       | 2241/9914 [23:26<1:11:27,  1.79it/s]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


 36%|███▋      | 3602/9914 [37:32<57:25,  1.83it/s]  

...

 63%|██████▎   | 6203/9914 [1:04:04<32:42,  1.89it/s]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0


 71%|███████   | 6998/9914 [1:12:04<26:42,  1.82it/s]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


 80%|███████▉  | 7918/9914 [1:21:24<20:34,  1.62it/s]

In [ ]:
timm.list_models()